In [48]:
import pandas as pd
import numpy as np

In [49]:
analys_record_personal_access = pd.read_csv('../../../../Data/For_Analysis_and_Modeling/1st_process/analys_record_personal_access.csv',
       sep='|',
       na_values=['',' ']
    )

In [50]:
analys_record_personal_access = analys_record_personal_access.drop(['des_acceso'], axis=1)

In [51]:
analys_record_personal_access.head()

,des_plan,anio_apertura_expediente,abandona,convocatoria_acceso,nota_admision_def,sexo,anio_nacimiento,provincia,municipio,facultad
0,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,N,JUN,7.16,H,1991,CÁCERES,CÁCERES,POLITECNICA
1,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,S,JUN,6.73,D,1991,BADAJOZ,BADAJOZ,POLITECNICA
2,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,S,JUN,8.10,H,1988,BADAJOZ,MÉRIDA,POLITECNICA
3,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,N,JUN,7.17,D,1991,CÁCERES,CORIA,POLITECNICA
4,GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIV...,2009-10,N,JUN,7.64,D,1991,CÁCERES,ALMOHARÍN,POLITECNICA


In [52]:
analys_record_personal_access_one_hot = pd.get_dummies(data=analys_record_personal_access, columns=analys_record_personal_access.drop(['nota_admision_def','abandona'], axis=1).columns)

In [53]:
analys_record_personal_access_one_hot.shape

(6406, 630)

In [54]:
analys_record_personal_access_one_hot.head()

,abandona,nota_admision_def,des_plan_GRADO EN EDIFICACIÓN,des_plan_GRADO EN EDUCACIÓN INFANTIL (CÁCERES),des_plan_GRADO EN EDUCACIÓN PRIMARIA (CÁCERES),des_plan_GRADO EN EDUCACIÓN PRIMARIA. Modalidad Bilingüe (Español-Inglés),des_plan_GRADO EN EDUCACIÓN SOCIAL,des_plan_GRADO EN INGENIERÍA CIVIL - CONSTRUCCIONES CIVILES,des_plan_GRADO EN INGENIERÍA CIVIL - HIDROLOGÍA,des_plan_GRADO EN INGENIERÍA CIVIL - TRANSPORTES Y SERVICIOS URBANOS,...,municipio_ZARZA DE GRANADILLA,municipio_ZARZA DE MONTÁNCHEZ,municipio_ZARZA LA MAYOR,municipio_ZORITA,municipio_ZUMAIA,municipio_ZUMARRAGA,municipio_ÁVILA,municipio_ÉCIJA,facultad_POLITECNICA,facultad_PROFESORADO
0,N,7.16,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,S,6.73,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,S,8.10,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,N,7.17,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,N,7.64,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [55]:
analys_record_personal_access_one_hot['abandona'].value_counts()

N    4680
S    1726
Name: abandona, dtype: int64

In [56]:
analys_record_personal_access_one_hot['abandona'] = analys_record_personal_access_one_hot['abandona'].apply(lambda func: 1 if func=='S' else 0)

In [57]:
analys_record_personal_access_one_hot['abandona'].value_counts()

0    4680
1    1726
Name: abandona, dtype: int64

In [58]:
from sklearn.utils import resample


In [59]:
# Separate majority and minority classes
df_majority = analys_record_personal_access_one_hot[analys_record_personal_access_one_hot.abandona==0]
df_minority = analys_record_personal_access_one_hot[analys_record_personal_access_one_hot.abandona==1]
 
# Downsample majority class
df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=1726,     # to match minority class
                                 random_state=123) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_majority_downsampled, df_minority])

In [60]:
df_downsampled.shape

(3452, 630)

In [61]:
df_downsampled.abandona.value_counts()

0    1726
1    1726
Name: abandona, dtype: int64

In [62]:
analys_record_personal_access_one_hot_modified = df_downsampled

In [63]:
X = analys_record_personal_access_one_hot_modified.drop(['abandona'], axis=1)
y = analys_record_personal_access_one_hot_modified['abandona']

In [64]:
from sklearn.model_selection import train_test_split, cross_val_score

In [65]:
import sklearn.metrics as sklm

In [66]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=24)

In [67]:
from sklearn.linear_model import LogisticRegression

In [68]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [69]:
def print_cv_results (cv_estimate):
    print('Mean performance metric = %4.3f' % np.mean(cv_estimate))
    print('SDT of the metric       = %4.3f' % np.std(cv_estimate))
    print('Outcomes by cv fold')
    for i, x in enumerate(cv_estimate):
        print('Fold %2d    %4.3f' % (i+1, x))

In [70]:
y_pred = clf.predict(X_test)
sklm.accuracy_score(y_true=y_test, y_pred=y_pred)

0.7485515643105446

In [71]:
sklm.confusion_matrix(y_test, y_pred)

array([[322, 113],
       [104, 324]])

In [72]:
tn, fp, fn, tp = sklm.confusion_matrix(y_test, y_pred).ravel()

In [73]:
sklm.recall_score(y_test, y_pred)

0.7570093457943925

In [74]:
tn/(tn+fp)

0.7402298850574712

Esta es la medida que tenemos que tratar de que sea la más alta, porque es preferible predecir un abandono como positivo para prevenirlo, y que luego realmente esa persona no vaya a abandonar, a que se nos escape un abandono.

El hecho de hacer que esta medida aumente, hará que el modelo predizca en más ocasiones un positivo en casos que no lo son, pero es preferible eso y que al modelo se le escapen los número de casos positivos menores posibles, a tratar de balancear los dos lados y que se nos escapen casos positivos.

Cuantos menos casos positivos se nos escapen, mejor, es preferible predecir un falso positivo que no un falso negativo

### Probamos con cv


In [75]:
print_cv_results (cross_val_score(clf, X_train, y_train))

/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/home/fran/.local/lib/python3.8/site

Mean performance metric = 0.764
SDT of the metric       = 0.023
Outcomes by cv fold
Fold  1    0.774
Fold  2    0.782
Fold  3    0.724
Fold  4    0.786
Fold  5    0.756


/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/home/fran/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



### Probamos con el cjto de datos completo

In [76]:
X_test = analys_record_personal_access_one_hot.drop(['abandona'], axis=1)
y_test = analys_record_personal_access_one_hot['abandona']

In [77]:
y_pred = clf.predict(X_test)
sklm.accuracy_score(y_true=y_test, y_pred=y_pred)

0.7706837339993756

In [78]:
sklm.confusion_matrix(y_test, y_pred)

array([[3544, 1136],
       [ 333, 1393]])

In [79]:
sklm.recall_score(y_test, y_pred)

0.8070683661645423

In [80]:
fp/(tn+fp)

0.2597701149425287

### Pasamos a ver la importancia de cada variable en el modelo

In [81]:
import time
from sklearn.inspection import permutation_importance


In [82]:
feature_names = X.columns

In [83]:
start_time = time.time()
result = permutation_importance(
    clf, X_test, y_test, n_repeats=3, random_state=42, n_jobs=-1)
elapsed_time = time.time() - start_time
print(f"Elapsed time to compute the importances: "
      f"{elapsed_time:.3f} seconds")

logit_importances = pd.Series(result.importances_mean, index=feature_names)

Elapsed time to compute the importances: 10.530 seconds


In [84]:
logit_importances = pd.Series(result.importances_mean, index=feature_names)

In [85]:
logit_importances = logit_importances[logit_importances > 0.001]

In [86]:
logit_importances.sort_values()

anio_nacimiento_1992                                                      0.001041
anio_nacimiento_1999                                                      0.001145
anio_apertura_expediente_2010-11                                          0.001145
anio_apertura_expediente_2014-15                                          0.001145
municipio_VALENCIA DE ALCÁNTARA                                           0.001301
sexo_H                                                                    0.001353
anio_nacimiento_2000                                                      0.001457
des_plan_GRADO EN EDUCACIÓN PRIMARIA (CÁCERES)                            0.001457
anio_nacimiento_1998                                                      0.001561
anio_apertura_expediente_2012-13                                          0.001977
anio_apertura_expediente_2016-17                                          0.002290
des_plan_GRADO EN INGENIERÍA CIVIL - HIDROLOGÍA                           0.002290
anio

In [87]:
logit_importances=pd.DataFrame({'Feature':logit_importances.sort_values(ascending=False).index,
              'Permutation_importance':logit_importances.sort_values(ascending=False)}).reset_index(drop=True)

In [88]:
logit_importances

,Feature,Permutation_importance
0,nota_admision_def,0.040587
1,facultad_POLITECNICA,0.026329
2,anio_apertura_expediente_2020-21,0.008742
3,facultad_PROFESORADO,0.007129
4,anio_nacimiento_1997,0.006140
5,provincia_CÁCERES,0.005984
6,anio_apertura_expediente_2011-12,0.005360
7,des_plan_GRADO EN INGENIERÍA INFORMÁTICA EN IN...,0.005308
8,anio_apertura_expediente_2017-18,0.003486
9,anio_nacimiento_1994,0.003434


In [90]:
import plotly.express as px


In [92]:
fig = px.bar(logit_importances, x='Feature', y='Permutation_importance')
fig.show()

In [44]:
def get_type_correlation(independent_feature:pd.Series, target_feature:pd.Series, positive_correlation:list, negative_correlation:list):
    from scipy import stats
    corr = stats.spearmanr(independent_feature,target_feature).correlation
    if corr > 0:
        positive_correlation.append(independent_feature.name)
    if corr < 0:
        negative_correlation.append(independent_feature.name)
        

In [45]:
positive_correlation = []
negative_correlation = []
for feature in logit_importances['Feature']:
    get_type_correlation(analys_record_personal_access_one_hot[feature], analys_record_personal_access_one_hot['abandona'], positive_correlation, negative_correlation)

In [46]:
positive_correlation

['facultad_POLITECNICA',
 'anio_apertura_expediente_2011-12',
 'des_plan_GRADO EN INGENIERÍA INFORMÁTICA EN INGENIERÍA DE COMPUTADORES',
 'anio_nacimiento_1994',
 'des_plan_GRADO EN INGENIERÍA CIVIL - HIDROLOGÍA',
 'anio_apertura_expediente_2012-13',
 'sexo_H',
 'municipio_VALENCIA DE ALCÁNTARA',
 'anio_apertura_expediente_2010-11',
 'anio_apertura_expediente_2014-15',
 'anio_nacimiento_1992']

In [47]:
negative_correlation

['nota_admision_def',
 'anio_apertura_expediente_2020-21',
 'facultad_PROFESORADO',
 'anio_nacimiento_1997',
 'provincia_CÁCERES',
 'anio_apertura_expediente_2017-18',
 'anio_nacimiento_1996',
 'anio_nacimiento_1995',
 'anio_apertura_expediente_2016-17',
 'anio_nacimiento_1998',
 'des_plan_GRADO EN EDUCACIÓN PRIMARIA (CÁCERES)',
 'anio_nacimiento_2000',
 'anio_nacimiento_1999']

La correlación positiva indica que si se cumple el hecho que se está examinando, aumenta la probabilidad de abandono.

La correlación negativa por el contrario, indica que si se cumple el hecho que se está examinando, disminuye la probabilidad de abandono.